In [2]:
%load_ext autoreload
%autoreload 2

import sys, os
sys.path.append(os.path.abspath(os.path.join('..')))

import numpy as np
import matplotlib.pyplot as plt 
import torch
import seaborn as sns
import numpy as np
from scipy.stats import pearsonr
import pandas as pd
import networkx as nx
from tqdm import tqdm_notebook as tqdm

from dpp.data.network import PPINetwork
from dpp.util import Params, prepare_sns
from dpp.data.associations import load_diseases
from dpp.experiments.protein_significance import ProteinSignificance

os.chdir("../")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Protein Significance

In [3]:
protein_sig_df = pd.read_csv("experiments/protein_significance/samples_1000/results.csv")

In [4]:
protein_sig_df.keys()

Index(['disease', 'protein', 'common_interactor_score',
       'common_interactor_score_pvalue', 'degree', 'frac_direct_interactions',
       'frac_direct_interactions_pvalue'],
      dtype='object')

In [28]:
ci_sig = protein_sig_df["common_interactor_score_pvalue"] < 0.05
di_sig = protein_sig_df["frac_direct_interactions_pvalue"] < 0.05 

In [34]:
ci_sig_di_nsig = np.logical_and(ci_sig, np.logical_not(di_sig))
ci_nsig_di_sig =  np.logical_and(di_sig, np.logical_not(ci_sig))

In [32]:
np.mean(ci_sig_di_nsig)

0.15384716941275878

In [35]:
np.mean(ci_nsig_di_sig)

0.021216201943388257

In [17]:
np.mean(di_sig)

0.17782266582171524

In [18]:
np.mean(ci_sig)

0.3104536332910858

In [36]:
np.mean(ci_sig) - np.mean(ci_sig_di_nsig)

0.156606463878327

In [37]:
np.mean(di_sig) - np.mean(ci_nsig_di_sig)

0.156606463878327

In [41]:
np.mean(protein_sig_df["frac_direct_interactions"] == 0)

0.46511934938741023

## DPP Performance

In [44]:
dpp_df = pd.read_csv("experiments/aggregation/results.csv", header=[0, 1])

In [59]:
# difference in recall-at-25
comparison = "rw"
(dpp_df["lci_dpp[DP]", "Recall-at-25"] - dpp_df[f"{comparison}_dpp[DP]", "Recall-at-25"] >= 0).mean()

0.7990060739922694

In [60]:
# difference in recall-at-25
comparison = "diamond"
(dpp_df["lci_dpp[DP]", "Recall-at-25"] - dpp_df[f"{comparison}_dpp[DP]", "Recall-at-25"] >= 0).mean()

0.8415240198785201

In [62]:
# difference in recall-at-25
comparison = "gcn-1layer"
(dpp_df["lci_dpp[DP]", "Recall-at-25"] - dpp_df[f"{comparison}_dpp[DP]", "Recall-at-25"] >= 0).mean()

KeyError: ('gcn-1layer_dpp[DP]', 'Recall-at-25')